In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.1:8b", temperature=0)

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [4]:
from datetime import datetime
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

current_date = datetime.now().strftime("%d/%m/%Y")

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt = f"""You are a travel agent. 
        Current Date: {current_date}
        
        When searching for flights, you must follow these rules strictly:
        1. DATE FORMAT: Use dd/mm/yyyy only.
        2. FUTURE DATES: All flight dates MUST be after {current_date}.
        3. NUMBERS: 'adults', 'children', 'infants', 'departureDateFlexRange' MUST be integers (numbers), not strings.
        4. CABIN: 'cabinClass' must be one of: 'M', 'W', 'C', 'F'. Default to 'M'.
        5. SORT: 'sort' must be one of: 'price', 'duration', 'quality', 'date'. Default to 'price'.
        6. LOCALE: 'locale' MUST be 'en'. 
        
        CRITICAL RULE FOR TOOL CALLS:
        - Do NOT include 'returnDate' in the tool call unless the user specifically requested a return flight. 
        - If a field is optional and you do not have a specific value for it, OMIT the key entirely from the tool call. Do not send empty strings ("").
        
        No follow up questions."""
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get me all the direct flights from San Francisco to Tokyo on March 31st")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to Tokyo on March 31st', additional_kwargs={}, response_metadata={}, id='89644d10-7f45-4e1c-a4c1-6d43a8afdccc'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-04T15:33:54.121349525Z', 'done': True, 'done_reason': 'stop', 'total_duration': 233207202485, 'load_duration': 15798476426, 'prompt_eval_count': 1364, 'prompt_eval_duration': 185137119980, 'eval_count': 83, 'eval_duration': 32064473297, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b89a1-304d-71b3-b117-b2bc81f3d92b-0', tool_calls=[{'name': 'search-flight', 'args': {'cabinClass': 'M', 'departureDate': '31/03/2026', 'flyFrom': 'San Francisco', 'flyTo': 'Tokyo', 'locale': 'en', 'passengers': {'adults': 1, 'children': 0, 'infants': 0}, 'sort': 'price'}, 'id': '48f0e60a-ecda-4583-859f-12d6f0e292c9', 'type': 'tool_call'}], usage_metadata=

In [7]:
print(response["messages"][-1].content)

This is a JSON array of objects, each representing a flight from San Francisco (SFO) to Tokyo (NRT or HND). Here's a breakdown of the structure:

* Each object has several properties:
	+ `flyFrom`: The departure airport code (always SFO).
	+ `flyTo`: The arrival airport code (either NRT or HND).
	+ `cityFrom` and `cityTo`: The city names for the departure and arrival airports.
	+ `departure`: An object with two properties:
		- `utc`: The UTC timestamp of the departure time.
		- `local`: The local timestamp of the departure time (in the format "HH:MM:SS").
	+ `arrival`: An object with two properties:
		- `utc`: The UTC timestamp of the arrival time.
		- `local`: The local timestamp of the arrival time (in the format "HH:MM:SS").
	+ `totalDurationInSeconds` and `durationInSeconds`: The total duration of the flight in seconds, and the duration without layovers, respectively.
	+ `price`: The price of the ticket.
	+ `deepLink`: A URL that can be used to book the ticket.
	+ `currency`: The c

In [8]:
print(response["messages"][-2].content)

[{'type': 'text', 'text': '[\n  {\n    "flyFrom": "SFO",\n    "flyTo": "NRT",\n    "cityFrom": "San Francisco",\n    "cityTo": "Tokyo",\n    "departure": {\n      "utc": "2026-03-31T23:45:00.000Z",\n      "local": "2026-03-31T16:45:00.000"\n    },\n    "arrival": {\n      "utc": "2026-04-01T11:00:00.000Z",\n      "local": "2026-04-01T20:00:00.000"\n    },\n    "totalDurationInSeconds": 40500,\n    "durationInSeconds": 40500,\n    "price": 330,\n    "deepLink": "https://on.kiwi.com/d6w196",\n    "currency": "EUR"\n  },\n  {\n    "flyFrom": "SFO",\n    "flyTo": "NRT",\n    "cityFrom": "San Francisco",\n    "cityTo": "Tokyo",\n    "departure": {\n      "utc": "2026-04-01T01:36:00.000Z",\n      "local": "2026-03-31T18:36:00.000"\n    },\n    "arrival": {\n      "utc": "2026-04-02T07:25:00.000Z",\n      "local": "2026-04-02T16:25:00.000"\n    },\n    "totalDurationInSeconds": 107340,\n    "durationInSeconds": 107340,\n    "price": 383,\n    "deepLink": "https://on.kiwi.com/GAxZhD",\n    "cu